In [0]:
import cv2
import numpy as np

In [37]:
# Part 2 of 8 - Affine Transformation
# Q3  

# Using the pseudo-inverse method from lecture 3-4

# Ax = B

# given n pairs of src & dst points, we have our A matrix to be a 2nx6 matrix
# given n pairs of src & dst points, we have our B matrix to be a 2nx1 matrix
# x is a 6x1 matrix which have the affine parameters (a1 - a6)

# here we have 7 set of corresponding points so, A is a 14x6 matrix and B is a
# 14x1 matrix

A = np.array([
  [0,11,1,0,0,0],
  [0,0,0,0,11,1],

  [0,0,1,0,0,0],
  [0,0,0,0,0,1],

  [6,0,1,0,0,0],
  [0,0,0,6,0,1],

  [6,11,1,0,0,0],
  [0,0,0,6,11,1],

  [3,16.5,1,0,0,0],
  [0,0,0,3,16.5,1],

  [0,11,1,0,0,0],
  [0,0,0,0,11,1],

  [6,11,1,0,0,0],
  [0,0,0,6,11,1],
])

B = np.array([
  [20],
  [12],
  [2],
  [0],
  [18],
  [4],
  [36],
  [16],
  [37],
  [20],
  [20],
  [12],
  [36],
  [16]
])

A_trans_A = A.T @ A # (A^T)A
A_trans_A_inv = np.linalg.inv(A_trans_A) # (A^TA)^-1
A_trans_B = A.T @ B # (A^T)B

x = A_trans_A_inv @ A_trans_B # (A^TA)^-1 @ (A^T)B

# x = np.array([
#   [a1],
#   [a2],
#   [a3],
#   [a4],
#   [a5],
#   [a6]
# ])

print(x)

[[ 2.66666667e+00]
 [ 1.63636364e+00]
 [ 2.00000000e+00]
 [ 6.66666667e-01]
 [ 1.09090909e+00]
 [-3.55271368e-15]]


In [38]:
# Part 3 of 8 - Axis of Rotation
# Q4

import numpy as np
import math

# crop the last column and the last row as they only have the translations
# we get a 3x3 matrix that has only the rotations in it.
# we take the svd of this Tr.
# we then no np.diag() on the d we recieved from the svd. The scale is around 1
# our final rotation matrix (3x3 because we took a 3x3 Tr) is the product of u and vh becasue they the 1st and 3rd rotations.
# we have to find the axis of rotation and angle of this final rotation
# 

tr = np.array([
  [0.9139, 0.0516, 0.4026],
  [0.0516, 0.9690, -0.2415],
  [-0.4026, 0.2415, 0.8829]
])

u,d,vh = np.linalg.svd(tr)

scale = np.diag(d)
# print('Scale : ', scale)

final_rotation_matrix = u @ vh

w, v = np.linalg.eig(final_rotation_matrix)
axis_rotation = v[:, 2]
print("Axis n: ", axis_rotation)

# trace of the rotation matrix = the sum of the diagonal elements of the rotation matrix
trace_rot_matrix = np.trace(final_rotation_matrix) # trace of the rotation matrix

cos_theeta = (trace_rot_matrix - 1) / 2
angle_theeta = math.acos(cos_theeta)
print('Angle Theta : ', angle_theeta) 

Axis n:  [ 5.14395927e-01+0.j  8.57552815e-01+0.j -2.34119019e-16+0.j]
Angle Theta :  0.4887065537227106


In [39]:
# Part 4 of 8 - Camera Model 1
# Q5

# y = -(fY/Z)

# f = 5,7,12,16,,24 (mm)
# Y = 1m = 1000mm
# Z = 93m = 93000mm

# sensors 
# 5.8x4.3mm -> 640x480 pixels 
# 7.2mmx5.3mm -> 1280x720 pixels or 1280x720 pixels 

# target must have at least 10x10 px dimension in image 

# we need to find a y such that when multipled with the pixels produced by a 
# particular lens, we get atleast 10ox which is the minimum that the target
# must have in the image

# we can check for all the focal lengths and the availible lens and choose the 
# one that gives the answer as 10px

# all in mm
Y = 1 * 1000
Z = 93 * 1000

focal_lengths = [5,7,12,16,24]

for x in focal_lengths:
  f = x
  y = -((f*Y)/Z)

  print("Focal Length: ", f, "mm")
  # sensor chosen: 5.8x4.3mm gives 640x480 pixels
  with_sen11 = y * (640/5.8) # 5.8_640
  with_sen12 = y * (480/4.3) # 4.3_480

  print("Sensor used 5.3x4.3mm gives 640x480 pixels: ", with_sen11, with_sen12)

  # sensor chosen: 7.2x5.3mm gives 1280x720 pixels
  with_sen21 = y * (1280/7.2) # 7.2_1280
  with_sen22 = y * (720/5.3) # 5.3_720

  print("Sensor used 7.2x5.3mm gives 1280x720 pixels: ", with_sen21, with_sen22)

  # sensor chosen: 7.2x5.3mm gives 1920x1080 pixels
  with_sen23 = y * (1920/7.2) # 7.2_1920
  with_sen24 = y * (1080/5.3) # 5.3_1080

  print("Sensor used 7.2x5.3mm gives 1920x1080 pixels: ", with_sen23, with_sen24)
  print("")

# from the results we can see that if we can see that shortest focal length of
# of a camera can be 5mm such that the image recognition can work because
# the target has at least 10x10px dimension in image.  


# Q6
# From the above answer we can see that the focal length of the camera, assuming 
# that the processor on board the drone can only process image of resolution 
# 640x480 in real-time, is 12mm as image recognition can work because
# the target has at least 10x10px dimension in image. This is because the answer
# is more that 10x10px.

Focal Length:  5 mm
Sensor used 5.3x4.3mm gives 640x480 pixels:  -5.932517612161662 -6.001500375093774
Sensor used 7.2x5.3mm gives 1280x720 pixels:  -9.557945041816009 -7.303712720632989
Sensor used 7.2x5.3mm gives 1920x1080 pixels:  -14.336917562724016 -10.955569080949484

Focal Length:  7 mm
Sensor used 5.3x4.3mm gives 640x480 pixels:  -8.305524657026327 -8.402100525131283
Sensor used 7.2x5.3mm gives 1280x720 pixels:  -13.381123058542414 -10.225197808886184
Sensor used 7.2x5.3mm gives 1920x1080 pixels:  -20.07168458781362 -15.337796713329277

Focal Length:  12 mm
Sensor used 5.3x4.3mm gives 640x480 pixels:  -14.238042269187988 -14.403600900225056
Sensor used 7.2x5.3mm gives 1280x720 pixels:  -22.939068100358423 -17.528910529519173
Sensor used 7.2x5.3mm gives 1920x1080 pixels:  -34.40860215053764 -26.29336579427876

Focal Length:  16 mm
Sensor used 5.3x4.3mm gives 640x480 pixels:  -18.984056358917318 -19.204801200300075
Sensor used 7.2x5.3mm gives 1280x720 pixels:  -30.58542413381123 

In [40]:
# Part 5 of 8 - Camera Model 2
# Q7 

P = np.array([ # perspective camera matrix
  [24, 40, 35, -2],
  [9, 34, 46, 31],
  [25, 49, 32, 29]
])

X = np.array([ # 3D point in homogeneous coordinates
  [2, 6, 5, 2]
])

X = X.T # take the transpose of X to match the matrix size when matrix multiplication

hx, hy, h = P @ X

u = hx/h
v = hy/h

print(u, v)

[0.81672598] [0.91459075]


In [41]:
# Part 6 of 8 - Epipolar Constraint
# Q8

# E = RS

T = [-0.3, 0.1, -9] # translation matrix
Tx = T[0]
Ty = T[1]
Tz = T[2]

S = np.array([ # the S matrix
  [0, -Tz, Ty],
  [Tz, 0, -Tx],
  [Tx, -Ty, 0]
])

R = np.array([ # rotation matrix along the z-axis with a rotation of 90 degree
  [0, 1, 0],
  [-1, 0, 0],
  [0, 0, 1]
])

E = R @ S
print("Essential Matrix = ")
print(E)

Essential Matrix = 
[[-9.   0.   0.3]
 [ 0.  -9.  -0.1]
 [-0.3 -0.1  0. ]]


In [42]:
# Q9
# F = K_inv_transpose x E x K_inv

K = np.array([ # intrinsic parameters matrix
  [8400, 0, 310],
  [0, 8300, 250],
  [0, 0, 1]
])

F = np.linalg.inv(K).T @ E @ np.linalg.inv(K) # the fundamental matrix

pl_bar = np.array([[25, 76, 1]])

pl_bar = pl_bar.T

coeff_epipole_line = F @ pl_bar

print("Coefficients of the epipolar line: ")
print("a: ", coeff_epipole_line[0])
print("b: ", coeff_epipole_line[1])
print("c: ", coeff_epipole_line[2])

Coefficients of the epipolar line: 
a:  [7.20663265e-05]
b:  [1.06836987e-05]
c:  [-0.01273653]


In [43]:
# Q10
# find the left epipole

Fu, Fd, Fv = np.linalg.svd(F)
Fv = Fv.T
left_epipole = Fv[:, 2]
left_epipole = left_epipole / left_epipole[2]

print("left-epipole: ", left_epipole)

left-epipole:  [822.50457354 545.84074853   1.        ]


In [44]:
# Q11
# slope of epipolar line in left image

xel_bar = left_epipole[0]
yel_bar = left_epipole[1]

x_left = 25
y_left = 76
m = (yel_bar - y_left)/(xel_bar - x_left)
print("Slope of the epipolar line in the left image: ", m)

Slope of the epipolar line in the left image:  0.5891386258002855


In [45]:
# Q12
# y-intercept of epipolar line in left image

c = y_left - m*x_left
print("y-intercept: ", c)

y-intercept:  61.271534354992866


In [46]:
# Q13
# right epipole

F_trans = F.T

FUp, FDp, FVp = np.linalg.svd(F_trans)
FVp = FVp.T

right_epipole = FVp[:, 2]
right_epipole = right_epipole / right_epipole[2]

print("right-epipole: ", right_epipole)

right-epipole:  [130.31397601 222.77378664   1.        ]


In [47]:
# Part 8 of 8 - 3D Convolution
# Q15
I = np.array(
    [
        [[0, 2, 3], [2, 2, 1], [3, 3, 3]],
        [[1, 1, 0], [0, 1, 1], [0, 0, 3]],
        [[2, 3, 1], [2, 3, 3], [1, 3, 1]]
    ]
)

F = np.array(
    [
        [[0, 1], [0, -1]],
        [[0, 0], [0, -1]],
        [[-1, -1], [-1, -1]]
    ]
)

output = np.array([[np.sum(I[:,0:2,0:2] * F), np.sum(I[:,0:2,1:3] * F)], # 2 x 2 x 1
                   [np.sum(I[:,1:3,0:2] * F), np.sum(I[:,1:3,1:3] * F)]])
print(output)

[[-11  -9]
 [-10 -15]]
